In [1]:
# Importing required libraries
!pip install wordcloud
import os
import time
import datetime
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tag import StanfordPOSTagger


from wordcloud import WordCloud

from pymongo import MongoClient
from pymongo import TEXT

import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('punkt')
nltk.download('stopwords')

import re

[nltk_data] Downloading package punkt to C:\Users\Alfredo
[nltk_data]     Salazar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Alfredo
[nltk_data]     Salazar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-bogota']
collection = db['elespectador-economia']
collection = db['elespectador-investigacion']
collection = db['elespectador-judicial']
collection = db['elespectador-politica']

In [3]:
text = []
for news in list(collection.find({}, {"title": 1, "summary": 1, "full_text": 1, "_id": 0})):
    text.append(news["title"])
    text.append(news["summary"])
    text.append(news["full_text"])

In [4]:
text = " ".join(text)

In [5]:
text

'“La Fiscalía no me da garantías”: Petro descarta, por ahora, renunciar al Congreso El candidato presidencial del Pacto Histórico se abstuvo de ahondar en la posible renuncia al Legislativo para centrarse en su campaña presidencial. En respuesta, el fiscal Francisco Barbosa dijo que “es lamentable que pretenda ser presidente de la república cabalgando sobre las instituciones judiciales”. Gustavo Petro, candidato presidencial del Pacto Histórico y aún senador de la República, se refirió este martes a las peticiones que le han hecho de que renuncie a su curul para dedicarse de lleno a la campaña que viene realizando, y que incluye varios recorridos por el país. A pesar de que han sido varios los sectores que le han hecho llamado al candidato, que hoy aparece como puntero en la mayoría de encuestas, Petro descartó el tema “por falta de garantías”. >Lea más sobre las elecciones de 2022 y otras noticias del mundo político “Ya veremos”, respondió Petro a la pregunta que le hizo una periodist

In [6]:
#At this point, it is necessary to be careful with warnings by losing some important words like names and others. 
text = text.lower()

In [7]:
#Tokenization per sentences.

sentences = sent_tokenize(text)

print(sentences)

['“la fiscalía no me da garantías”: petro descarta, por ahora, renunciar al congreso el candidato presidencial del pacto histórico se abstuvo de ahondar en la posible renuncia al legislativo para centrarse en su campaña presidencial.', 'en respuesta, el fiscal francisco barbosa dijo que “es lamentable que pretenda ser presidente de la república cabalgando sobre las instituciones judiciales”.', 'gustavo petro, candidato presidencial del pacto histórico y aún senador de la república, se refirió este martes a las peticiones que le han hecho de que renuncie a su curul para dedicarse de lleno a la campaña que viene realizando, y que incluye varios recorridos por el país.', 'a pesar de que han sido varios los sectores que le han hecho llamado al candidato, que hoy aparece como puntero en la mayoría de encuestas, petro descartó el tema “por falta de garantías”.', '>lea más sobre las elecciones de 2022 y otras noticias del mundo político “ya veremos”, respondió petro a la pregunta que le hizo 

In [8]:
#Tokenization per words with removing punctuation and empty spaces.

tokenizer = nltk.RegexpTokenizer(r"\w+")

words = tokenizer.tokenize(text)

print(words)

['la', 'fiscalía', 'no', 'me', 'da', 'garantías', 'petro', 'descarta', 'por', 'ahora', 'renunciar', 'al', 'congreso', 'el', 'candidato', 'presidencial', 'del', 'pacto', 'histórico', 'se', 'abstuvo', 'de', 'ahondar', 'en', 'la', 'posible', 'renuncia', 'al', 'legislativo', 'para', 'centrarse', 'en', 'su', 'campaña', 'presidencial', 'en', 'respuesta', 'el', 'fiscal', 'francisco', 'barbosa', 'dijo', 'que', 'es', 'lamentable', 'que', 'pretenda', 'ser', 'presidente', 'de', 'la', 'república', 'cabalgando', 'sobre', 'las', 'instituciones', 'judiciales', 'gustavo', 'petro', 'candidato', 'presidencial', 'del', 'pacto', 'histórico', 'y', 'aún', 'senador', 'de', 'la', 'república', 'se', 'refirió', 'este', 'martes', 'a', 'las', 'peticiones', 'que', 'le', 'han', 'hecho', 'de', 'que', 'renuncie', 'a', 'su', 'curul', 'para', 'dedicarse', 'de', 'lleno', 'a', 'la', 'campaña', 'que', 'viene', 'realizando', 'y', 'que', 'incluye', 'varios', 'recorridos', 'por', 'el', 'país', 'a', 'pesar', 'de', 'que', 'han

In [9]:
#Stopwords list for spanish language.

stop_words = set(stopwords.words("spanish"))
print(stop_words)

{'teníais', 'hubiesen', 'habían', 'poco', 'esas', 'habrán', 'unos', 'estuvieran', 'un', 'tengamos', 'seremos', 'estuviera', 'sin', 'estáis', 'tuviera', 'han', 'qué', 'tuviésemos', 'habiendo', 'suyas', 'tengan', 'estoy', 'ni', 'esto', 'vuestro', 'estuviésemos', 'estaríais', 'estábamos', 'era', 'entre', 'están', 'estuvieseis', 'esté', 'tenías', 'tuvo', 'antes', 'mi', 'eras', 'vuestra', 'hasta', 'estarán', 'una', 'habríamos', 'estad', 'estuviesen', 'siente', 'donde', 'mío', 'hayan', 'tenemos', 'soy', 'tuviese', 'tenidas', 'mías', 'para', 'tú', 'cual', 'habrían', 'estaban', 'estaríamos', 'tuyo', 'la', 'habremos', 'hubierais', 'del', 'tienes', 'tuviste', 'estarás', 'estabais', 'hubiese', 'hayamos', 'tuviéramos', 'ellos', 'de', 'algo', 'tuyas', 'tenida', 'estemos', 'fuiste', 'todo', 'estuvieron', 'fuera', 'seamos', 'será', 'porque', 'y', 'está', 'las', 'no', 'fuéramos', 'nosotros', 'fuese', 'tenidos', 'e', 'tendremos', 'estabas', 'otro', 'hubiésemos', 'mis', 'me', 'tengo', 'fueras', 'tiene',

In [10]:
#Removing Stopwords.

filtered_words = []
for w in words:
    if w not in stop_words:
        filtered_words.append(w)

print("All words:", words)
print("\n")
print("Substracting stopwords:", filtered_words)

All words: ['la', 'fiscalía', 'no', 'me', 'da', 'garantías', 'petro', 'descarta', 'por', 'ahora', 'renunciar', 'al', 'congreso', 'el', 'candidato', 'presidencial', 'del', 'pacto', 'histórico', 'se', 'abstuvo', 'de', 'ahondar', 'en', 'la', 'posible', 'renuncia', 'al', 'legislativo', 'para', 'centrarse', 'en', 'su', 'campaña', 'presidencial', 'en', 'respuesta', 'el', 'fiscal', 'francisco', 'barbosa', 'dijo', 'que', 'es', 'lamentable', 'que', 'pretenda', 'ser', 'presidente', 'de', 'la', 'república', 'cabalgando', 'sobre', 'las', 'instituciones', 'judiciales', 'gustavo', 'petro', 'candidato', 'presidencial', 'del', 'pacto', 'histórico', 'y', 'aún', 'senador', 'de', 'la', 'república', 'se', 'refirió', 'este', 'martes', 'a', 'las', 'peticiones', 'que', 'le', 'han', 'hecho', 'de', 'que', 'renuncie', 'a', 'su', 'curul', 'para', 'dedicarse', 'de', 'lleno', 'a', 'la', 'campaña', 'que', 'viene', 'realizando', 'y', 'que', 'incluye', 'varios', 'recorridos', 'por', 'el', 'país', 'a', 'pesar', 'de', 

In [11]:
filtered_words = []
for i in words : 
    filtered_words += nltk.word_tokenize(i)
print(filtered_words)

['la', 'fiscalía', 'no', 'me', 'da', 'garantías', 'petro', 'descarta', 'por', 'ahora', 'renunciar', 'al', 'congreso', 'el', 'candidato', 'presidencial', 'del', 'pacto', 'histórico', 'se', 'abstuvo', 'de', 'ahondar', 'en', 'la', 'posible', 'renuncia', 'al', 'legislativo', 'para', 'centrarse', 'en', 'su', 'campaña', 'presidencial', 'en', 'respuesta', 'el', 'fiscal', 'francisco', 'barbosa', 'dijo', 'que', 'es', 'lamentable', 'que', 'pretenda', 'ser', 'presidente', 'de', 'la', 'república', 'cabalgando', 'sobre', 'las', 'instituciones', 'judiciales', 'gustavo', 'petro', 'candidato', 'presidencial', 'del', 'pacto', 'histórico', 'y', 'aún', 'senador', 'de', 'la', 'república', 'se', 'refirió', 'este', 'martes', 'a', 'las', 'peticiones', 'que', 'le', 'han', 'hecho', 'de', 'que', 'renuncie', 'a', 'su', 'curul', 'para', 'dedicarse', 'de', 'lleno', 'a', 'la', 'campaña', 'que', 'viene', 'realizando', 'y', 'que', 'incluye', 'varios', 'recorridos', 'por', 'el', 'país', 'a', 'pesar', 'de', 'que', 'han

In [12]:
tokens = nltk.pos_tag(filtered_words)
print(tokens)

[('la', 'NN'), ('fiscalía', 'VBZ'), ('no', 'DT'), ('me', 'PRP'), ('da', 'VBZ'), ('garantías', 'JJ'), ('petro', 'NN'), ('descarta', 'NN'), ('por', 'NN'), ('ahora', 'NN'), ('renunciar', 'NN'), ('al', 'NN'), ('congreso', 'NN'), ('el', 'NN'), ('candidato', 'VBP'), ('presidencial', 'JJ'), ('del', 'NN'), ('pacto', 'NN'), ('histórico', 'NN'), ('se', 'FW'), ('abstuvo', 'FW'), ('de', 'FW'), ('ahondar', 'FW'), ('en', 'FW'), ('la', 'FW'), ('posible', 'JJ'), ('renuncia', 'NN'), ('al', 'NN'), ('legislativo', 'NN'), ('para', 'NN'), ('centrarse', 'NN'), ('en', 'FW'), ('su', 'NN'), ('campaña', 'VBP'), ('presidencial', 'JJ'), ('en', 'FW'), ('respuesta', 'NN'), ('el', 'JJ'), ('fiscal', 'JJ'), ('francisco', 'NN'), ('barbosa', 'NN'), ('dijo', 'NN'), ('que', 'NN'), ('es', 'NN'), ('lamentable', 'JJ'), ('que', 'NN'), ('pretenda', 'NN'), ('ser', 'NN'), ('presidente', 'NN'), ('de', 'IN'), ('la', 'FW'), ('república', 'FW'), ('cabalgando', 'NN'), ('sobre', 'NN'), ('las', 'NN'), ('instituciones', 'NNS'), ('judici

In [13]:
frec_tokens = FreqDist(t)

print(tokens)

<FreqDist with 1184 samples and 4132 outcomes>


In [14]:
frec_tokens.most_common(20)

NameError: name 'frec_tokens' is not defined

In [ ]:
frec_dist = FreqDist(filtered_words)

print(frec_dist)

In [ ]:
frec_dist.most_common(20)

In [ ]:
n = 10
plt.figure(figsize=(30,10))
plt.bar([ w[0] for w in frec_dist.most_common(n) ], [ w[1] for w in frec_dist.most_common(n) ])

In [ ]:
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = "black").generate(" ".join(filtered_words))

plt.imshow(wordcloud, interpolation = "bilinear")
plt.axis("off")
plt.show()